# DeepFactor - Experiments

### Deep Factor (Wang et al.,  2019)

Hybrid method that uses *local* methods that are specific to each time series in the dataset as well as a *global* representation trained on the entire data. The latter is a RNN to learn universal pattern. The current version uses another RNN as local model -> **DeepFactor-RNN (DF-RNN)**

- [`DeepFactorEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deep_factor.html?highlight=deepfactorestimator#gluonts.model.deep_factor.DeepFactorEstimator)
- [Wang et al. (2019)](https://arxiv.org/abs/1905.12417)

In [1]:
# if version does not include DeepState, uncomment the following:
!pip install git+https://github.com/awslabs/gluon-ts.git

import gluonts
print(gluonts.__version__)

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# gluon distribution
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.distribution.gaussian import GaussianOutput

# model imports 
# from gluonts.model.deepstate import DeepStateEstimator
from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

  Cloning https://github.com/awslabs/gluon-ts.git to /tmp/pip-req-build-9ym96yed
  Installing build dependencies ... done
    100% |████████████████████████████████| 81kB 5.9MB/s ta 0:00:011
    100% |████████████████████████████████| 28.4MB 1.7MB/s eta 0:00:01
    100% |████████████████████████████████| 10.4MB 8.4MB/s eta 0:00:01
    100% |████████████████████████████████| 4.8MB 13.4MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 31.0MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 40.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for gluonts ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-6cd48l43/wheels/09/ba/b9/c830628df4d977a9dfe661f6001cdb861fc3f2524bac0ee433
  Running setup.py bdist_wheel for holidays ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/40/a9/2b/94ac5464363d37564a87dc93a9d21a5850aac14a4608197003
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/28/77

INFO:root:Using CPU


In [2]:
def main(seed=42, data="m4_daily", epochs=100, batches=50):
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    dataset = get_dataset(data)
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepFactorEstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         num_layers_global=2,
#         num_layers_local=2
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}

    pprint(output)

# if __name__ == "__main__":
#     simple_main()

In [37]:
%%time
if __name__ == "__main__":
    main(seed=42, data="m4_daily", epochs=100, batches=50)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 20.90it/s, avg_epoch_loss=6.26e+7]
INFO:root:Epoch[0] Elapsed time 2.396 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=62617162.011429
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.14it/s, avg_epoch_loss=4.6e+7]
INFO:root:Epoch[1] Elapsed time 2.260 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=46036213.371429
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 25.85it/s, avg_epoch_loss=2.18e+7]
INFO:root:Epoch[2] Elapsed time 1.937 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=21781244.925714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.31it/s, avg_epoch_loss=1.3e+7]
INFO:root:Epoch[3] Elapsed time 2.244 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=12953914.782857
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [0

{'MASE': 39.81111173,
 'MSIS': 1532.61294171,
 'sMAPE': 0.29508529,
 'wQuantileLoss[0.5]': 0.30571222,
 'wQuantileLoss[0.9]': 0.28655776}
CPU times: user 10min 21s, sys: 5.36 s, total: 10min 26s
Wall time: 10min 27s


In [38]:
%%time
if __name__ == "__main__":
    main(seed=43, data="m4_daily", epochs=100, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.26it/s, avg_epoch_loss=6.04e+7]
INFO:root:Epoch[0] Elapsed time 2.358 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=60432346.285714
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.52it/s, avg_epoch_loss=4.27e+7]
INFO:root:Epoch[1] Elapsed time 2.224 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=42716719.428571
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 25.98it/s, avg_epoch_loss=2.04e+7]
INFO:root:Epoch[2] Elapsed time 1.928 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=20431871.325714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.43it/s, avg_epoch_loss=1.28e+7]
INFO:root:Epoch[3] Elapsed time 2.233 seconds
INFO:root:Epoch[3] Evaluation m

{'MASE': 56.81531908,
 'MSIS': 2221.36697262,
 'sMAPE': 0.32490977,
 'wQuantileLoss[0.5]': 0.49594255,
 'wQuantileLoss[0.9]': 0.15688109}
CPU times: user 9min 59s, sys: 4.53 s, total: 10min 3s
Wall time: 9min 56s


In [39]:
%%time
if __name__ == "__main__":
    main(seed=44, data="m4_daily", epochs=100, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.34it/s, avg_epoch_loss=6.6e+7]
INFO:root:Epoch[0] Elapsed time 2.353 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=66045109.028571
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.48it/s, avg_epoch_loss=5.55e+7]
INFO:root:Epoch[1] Elapsed time 2.226 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=55483667.337143
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 25.86it/s, avg_epoch_loss=3.03e+7]
INFO:root:Epoch[2] Elapsed time 1.939 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=30276762.251429
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.22it/s, avg_epoch_loss=2.1e+7]
INFO:root:Epoch[3] Elapsed time 2.254 seconds
INFO:root:Epoch[3] Evaluation met

{'MASE': 63.97786994,
 'MSIS': 2501.54593099,
 'sMAPE': 0.36858386,
 'wQuantileLoss[0.5]': 0.5379713,
 'wQuantileLoss[0.9]': 0.19956738}
CPU times: user 10min 5s, sys: 4.32 s, total: 10min 9s
Wall time: 10min 3s


In [40]:
%%time
if __name__ == "__main__":
    main(seed=42, data="m4_daily", epochs=150, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.28it/s, avg_epoch_loss=6.26e+7]
INFO:root:Epoch[0] Elapsed time 2.354 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=62617162.011429
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.53it/s, avg_epoch_loss=4.6e+7]
INFO:root:Epoch[1] Elapsed time 2.221 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=46036213.371429
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 25.89it/s, avg_epoch_loss=2.18e+7]
INFO:root:Epoch[2] Elapsed time 1.934 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=21781244.925714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.39it/s, avg_epoch_loss=1.3e+7]
INFO:root:Epoch[3] Elapsed time 2.239 seconds
INFO:root:Epoch[3] Evaluation met

{'MASE': 33.21698268,
 'MSIS': 1245.23082926,
 'sMAPE': 0.29413541,
 'wQuantileLoss[0.5]': 0.25918701,
 'wQuantileLoss[0.9]': 0.41749721}
CPU times: user 13min 48s, sys: 6.32 s, total: 13min 54s
Wall time: 13min 45s


In [41]:
%%time
if __name__ == "__main__":
    main(seed=43, data="m4_daily", epochs=150, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  3.87it/s, avg_epoch_loss=6.04e+7]
INFO:root:Epoch[0] Elapsed time 12.917 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=60432346.285714
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:07<00:00,  6.59it/s, avg_epoch_loss=4.27e+7]
INFO:root:Epoch[1] Elapsed time 7.590 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=42716719.428571
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:05<00:00,  8.59it/s, avg_epoch_loss=2.04e+7]
INFO:root:Epoch[2] Elapsed time 5.825 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=20431871.325714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  7.53it/s, avg_epoch_loss=1.28e+7]
INFO:root:Epoch[3] Elapsed time 6.639 seconds
INFO:root:Epoch[3] Evaluation 

{'MASE': 29.89307805,
 'MSIS': 1137.67781581,
 'sMAPE': 0.23886131,
 'wQuantileLoss[0.5]': 0.23515806,
 'wQuantileLoss[0.9]': 0.27140829}
CPU times: user 13min 41s, sys: 6.37 s, total: 13min 47s
Wall time: 13min 41s


In [6]:
%%time
if __name__ == "__main__":
    main(seed=44, data="m4_daily", epochs=150, batches=50)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 18.39it/s, avg_epoch_loss=6.6e+7]
INFO:root:Epoch[0] Elapsed time 2.722 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=66045109.028571
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.77it/s, avg_epoch_loss=5.55e+7]
INFO:root:Epoch[1] Elapsed time 2.300 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=55483667.337143
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 25.19it/s, avg_epoch_loss=3.03e+7]
INFO:root:Epoch[2] Elapsed time 1.988 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=30276762.251429
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.89it/s, avg_epoch_loss=2.1e+7]
INFO:root:Epoch[3] Elapsed time 2.287 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=21032375.702857
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [0

{'MASE': 51.05314329,
 'MSIS': 1959.59859923,
 'sMAPE': 0.30559857,
 'wQuantileLoss[0.5]': 0.40463466,
 'wQuantileLoss[0.9]': 0.12930256}
CPU times: user 8min 10s, sys: 4.91 s, total: 8min 15s
Wall time: 8min 8s


In [7]:
%%time
if __name__ == "__main__":
    main(seed=42, data="m4_daily", epochs=200, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 20.93it/s, avg_epoch_loss=6.26e+7]
INFO:root:Epoch[0] Elapsed time 2.392 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=62617162.011429
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.23it/s, avg_epoch_loss=4.6e+7]
INFO:root:Epoch[1] Elapsed time 2.253 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=46036213.371429
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 24.49it/s, avg_epoch_loss=2.18e+7]
INFO:root:Epoch[2] Elapsed time 2.045 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=21781244.925714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.93it/s, avg_epoch_loss=1.3e+7]
INFO:root:Epoch[3] Elapsed time 2.285 seconds
INFO:root:Epoch[3] Evaluation met

{'MASE': 16.05410615,
 'MSIS': 552.52366906,
 'sMAPE': 0.12951341,
 'wQuantileLoss[0.5]': 0.12731044,
 'wQuantileLoss[0.9]': 0.12750281}
CPU times: user 11min 23s, sys: 6.1 s, total: 11min 29s
Wall time: 11min 12s


In [8]:
%%time
if __name__ == "__main__":
    main(seed=43, data="m4_daily", epochs=200, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 20.83it/s, avg_epoch_loss=6.04e+7]
INFO:root:Epoch[0] Elapsed time 2.402 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=60432346.285714
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.34it/s, avg_epoch_loss=4.27e+7]
INFO:root:Epoch[1] Elapsed time 2.345 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=42716719.428571
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 24.38it/s, avg_epoch_loss=2.04e+7]
INFO:root:Epoch[2] Elapsed time 2.053 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=20431871.325714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 18.55it/s, avg_epoch_loss=1.28e+7]
INFO:root:Epoch[3] Elapsed time 2.699 seconds
INFO:root:Epoch[3] Evaluation m

{'MASE': 20.0279132,
 'MSIS': 746.12020351,
 'sMAPE': 0.14278343,
 'wQuantileLoss[0.5]': 0.17515089,
 'wQuantileLoss[0.9]': 0.07639333}
CPU times: user 19min 41s, sys: 9.43 s, total: 19min 51s
Wall time: 19min 39s


In [9]:
%%time
if __name__ == "__main__":
    main(seed=44, data="m4_daily", epochs=200, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 20.87it/s, avg_epoch_loss=6.6e+7]
INFO:root:Epoch[0] Elapsed time 2.399 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=66045109.028571
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.03it/s, avg_epoch_loss=5.55e+7]
INFO:root:Epoch[1] Elapsed time 2.273 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=55483667.337143
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 25.33it/s, avg_epoch_loss=3.03e+7]
INFO:root:Epoch[2] Elapsed time 1.978 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=30276762.251429
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.81it/s, avg_epoch_loss=2.1e+7]
INFO:root:Epoch[3] Elapsed time 2.295 seconds
INFO:root:Epoch[3] Evaluation met

{'MASE': 16.55292226,
 'MSIS': 574.24439425,
 'sMAPE': 0.12937567,
 'wQuantileLoss[0.5]': 0.13081623,
 'wQuantileLoss[0.9]': 0.12079728}
CPU times: user 18min 53s, sys: 8.62 s, total: 19min 2s
Wall time: 18min 51s


In [10]:
%%time
if __name__ == "__main__":
    main(seed=42, data="m4_daily", epochs=250, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:07<00:00,  7.11it/s, avg_epoch_loss=6.26e+7]
INFO:root:Epoch[0] Elapsed time 7.056 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=62617162.011429
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  7.40it/s, avg_epoch_loss=4.6e+7]
INFO:root:Epoch[1] Elapsed time 6.780 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=46036213.371429
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:05<00:00,  8.41it/s, avg_epoch_loss=2.18e+7]
INFO:root:Epoch[2] Elapsed time 5.947 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=21781244.925714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  7.31it/s, avg_epoch_loss=1.3e+7]
INFO:root:Epoch[3] Elapsed time 6.841 seconds
INFO:root:Epoch[3] Evaluation met

{'MASE': 11.09261112,
 'MSIS': 363.60744176,
 'sMAPE': 0.09054291,
 'wQuantileLoss[0.5]': 0.09058573,
 'wQuantileLoss[0.9]': 0.05056335}
CPU times: user 21min 30s, sys: 9.97 s, total: 21min 40s
Wall time: 21min 29s


In [11]:
%%time
if __name__ == "__main__":
    main(seed=43, data="m4_daily", epochs=250, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 20.34it/s, avg_epoch_loss=6.04e+7]
INFO:root:Epoch[0] Elapsed time 2.462 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=60432346.285714
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.13it/s, avg_epoch_loss=4.27e+7]
INFO:root:Epoch[1] Elapsed time 2.368 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=42716719.428571
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.60it/s, avg_epoch_loss=2.04e+7]
INFO:root:Epoch[2] Elapsed time 2.215 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=20431871.325714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 18.22it/s, avg_epoch_loss=1.28e+7]
INFO:root:Epoch[3] Elapsed time 2.746 seconds
INFO:root:Epoch[3] Evaluation m

{'MASE': 11.09508159,
 'MSIS': 389.24279887,
 'sMAPE': 0.09089525,
 'wQuantileLoss[0.5]': 0.09249199,
 'wQuantileLoss[0.9]': 0.12978544}
CPU times: user 23min 17s, sys: 10.2 s, total: 23min 27s
Wall time: 23min 14s


In [12]:
%%time
if __name__ == "__main__":
    main(seed=44, data="m4_daily", epochs=250, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:07<00:00,  6.81it/s, avg_epoch_loss=6.6e+7]
INFO:root:Epoch[0] Elapsed time 7.358 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=66045109.028571
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  7.41it/s, avg_epoch_loss=5.55e+7]
INFO:root:Epoch[1] Elapsed time 6.764 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=55483667.337143
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  8.31it/s, avg_epoch_loss=3.03e+7]
INFO:root:Epoch[2] Elapsed time 6.034 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=30276762.251429
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:07<00:00,  7.13it/s, avg_epoch_loss=2.1e+7]
INFO:root:Epoch[3] Elapsed time 7.012 seconds
INFO:root:Epoch[3] Evaluation met

{'MASE': 8.50800101,
 'MSIS': 263.82989985,
 'sMAPE': 0.0712435,
 'wQuantileLoss[0.5]': 0.07082789,
 'wQuantileLoss[0.9]': 0.04476683}
CPU times: user 21min 40s, sys: 10.4 s, total: 21min 50s
Wall time: 21min 40s


In [13]:
%%time
if __name__ == "__main__":
    main(seed=42, data="m4_daily", epochs=300, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 20.51it/s, avg_epoch_loss=6.26e+7]
INFO:root:Epoch[0] Elapsed time 2.442 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=62617162.011429
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.17it/s, avg_epoch_loss=4.6e+7]
INFO:root:Epoch[1] Elapsed time 2.364 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=46036213.371429
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.76it/s, avg_epoch_loss=2.18e+7]
INFO:root:Epoch[2] Elapsed time 2.302 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=21781244.925714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 18.02it/s, avg_epoch_loss=1.3e+7]
INFO:root:Epoch[3] Elapsed time 2.777 seconds
INFO:root:Epoch[3] Evaluation met

{'MASE': 10.86580343,
 'MSIS': 351.00028568,
 'sMAPE': 0.08830235,
 'wQuantileLoss[0.5]': 0.09287248,
 'wQuantileLoss[0.9]': 0.03429035}
CPU times: user 26min 46s, sys: 12.8 s, total: 26min 59s
Wall time: 26min 46s


In [3]:
%%time
if __name__ == "__main__":
    main(seed=43, data="m4_daily", epochs=300, batches=50)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 18.84it/s, avg_epoch_loss=6.04e+7]
INFO:root:Epoch[0] Elapsed time 2.659 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=60432346.285714
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.98it/s, avg_epoch_loss=4.27e+7]
INFO:root:Epoch[1] Elapsed time 2.277 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=42716719.428571
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 25.35it/s, avg_epoch_loss=2.04e+7]
INFO:root:Epoch[2] Elapsed time 1.974 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=20431871.325714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 22.21it/s, avg_epoch_loss=1.28e+7]
INFO:root:Epoch[3] Elapsed time 2.253 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=12786790.297143
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 

{'MASE': 10.88786361,
 'MSIS': 379.82270993,
 'sMAPE': 0.08810996,
 'wQuantileLoss[0.5]': 0.08533324,
 'wQuantileLoss[0.9]': 0.08414677}
CPU times: user 24min 21s, sys: 12.5 s, total: 24min 33s
Wall time: 24min 19s


In [4]:
%%time
if __name__ == "__main__":
    main(seed=44, data="m4_daily", epochs=300, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:07<00:00,  6.78it/s, avg_epoch_loss=6.6e+7]
INFO:root:Epoch[0] Elapsed time 7.381 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=66045109.028571
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  7.32it/s, avg_epoch_loss=5.55e+7]
INFO:root:Epoch[1] Elapsed time 6.833 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=55483667.337143
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  8.13it/s, avg_epoch_loss=3.03e+7]
INFO:root:Epoch[2] Elapsed time 6.149 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=30276762.251429
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  7.23it/s, avg_epoch_loss=2.1e+7]
INFO:root:Epoch[3] Elapsed time 6.921 seconds
INFO:root:Epoch[3] Evaluation met

{'MASE': 9.02917417,
 'MSIS': 286.6056074,
 'sMAPE': 0.07456465,
 'wQuantileLoss[0.5]': 0.07721351,
 'wQuantileLoss[0.9]': 0.03572687}
CPU times: user 26min 53s, sys: 13.1 s, total: 27min 7s
Wall time: 26min 56s


In [ ]:
%%time
if __name__ == "__main__":
    main(seed=42, data="m4_daily", epochs=350, batches=50)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 19.71it/s, avg_epoch_loss=6.26e+7]
INFO:root:Epoch[0] Elapsed time 2.549 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=62617162.011429
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 18.23it/s, avg_epoch_loss=4.6e+7]
INFO:root:Epoch[1] Elapsed time 2.752 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=46036213.371429
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 18.72it/s, avg_epoch_loss=2.18e+7]
INFO:root:Epoch[2] Elapsed time 2.695 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=21781244.925714
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  7.26it/s, avg_epoch_loss=1.3e+7]
INFO:root:Epoch[3] Elapsed time 6.892 seconds
INFO:root:Epoch[3] Evaluation met

In [ ]:
%%time
if __name__ == "__main__":
    main(seed=43, data="m4_daily", epochs=350, batches=50)

In [ ]:
%%time
if __name__ == "__main__":
    main(seed=44, data="m4_daily", epochs=350, batches=50)

In [ ]:
%%time
if __name__ == "__main__":
    main(seed=42, data="m4_daily", epochs=400, batches=50)

In [ ]:
%%time
if __name__ == "__main__":
    main(seed=43, data="m4_daily", epochs=400, batches=50)

In [ ]:
%%time
if __name__ == "__main__":
    main(seed=44, data="m4_daily", epochs=400, batches=50)